<a href="https://colab.research.google.com/github/mahb97/fw-vs-ulysses-zipf/blob/main/fit_zipf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path
import numpy as np, pandas as pd
import regex as re
from collections import Counter
from dataclasses import dataclass
from typing import Dict, Tuple, List
import math, json, warnings

# Colab / Drive
IN_COLAB = False
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    pass

if IN_COLAB:
    from google.colab import drive, files
    drive.mount('/content/drive', force_remount=True)
    ROOT = Path("/content/drive/MyDrive/zipf_joyce").resolve()
else:
    ROOT = (Path.cwd() / "zipf_joyce_local").resolve()

DATA_PROC = ROOT / "data" / "processed"
RESULTS   = ROOT / "results"
TABLES    = RESULTS / "tables"
FIGS      = RESULTS / "figures"
for p in (TABLES, FIGS): p.mkdir(parents=True, exist_ok=True)

print("ROOT:", ROOT)
print("Processed:", DATA_PROC)
print("Tables:", TABLES)

Mounted at /content/drive
ROOT: /content/drive/MyDrive/zipf_joyce
Processed: /content/drive/MyDrive/zipf_joyce/data/processed
Tables: /content/drive/MyDrive/zipf_joyce/results/tables


In [2]:
import regex as re, unicodedata as ud

# Redefine CONFIGS with an explicit flag instead of relying on object identity
CONFIGS = {
    "conservative": dict(lower=False, nfkc=True, split_hyphens=False, remove_digits=False, min_len=1, fw_split_camel=False),
    "canonical":    dict(lower=True,  nfkc=True, split_hyphens=True,  remove_digits=True,  min_len=1, fw_split_camel=False),
    "generous":     dict(lower=True,  nfkc=True, split_hyphens=True,  remove_digits=True,  min_len=1, fw_split_camel=True),
}

# Rebind tokenize()
WORD_RE = re.compile(r"\p{L}[\p{L}\p{M}\p{Pd}\p{Pc}\']*")

def tokenize(text: str, cfg: dict):
    s = text
    if cfg.get("nfkc", True):
        s = ud.normalize("NFKC", s)
    if cfg.get("lower", True):
        s = s.lower()
    if cfg.get("remove_digits", False):
        s = re.sub(r"\p{N}+", " ", s)
    toks = WORD_RE.findall(s)
    if cfg.get("split_hyphens", False):
        split = []
        for t in toks:
            split.extend(t.split("-"))
        toks = [t for t in split if t]
    if cfg.get("fw_split_camel", False):
        # Split lower→Upper seams
        g = []
        for t in toks:
            g.extend(re.sub(r"(?<=[\p{Ll}])(?=[\p{Lu}])", " ", t).split())
        toks = g
    mlen = cfg.get("min_len", 1)
    return [t for t in toks if len(t) >= mlen]

In [3]:
# Three tokenisation tracks
CONFIGS = {
    "conservative": dict(lower=False, nfkc=True, split_hyphens=False, remove_digits=False, min_len=1),
    "canonical":    dict(lower=True,  nfkc=True, split_hyphens=True,  remove_digits=True,  min_len=1),
    "generous":     dict(lower=True,  nfkc=True, split_hyphens=True,  remove_digits=True,  min_len=1), # FW-aware extras below
}

WORD_RE = re.compile(r"\p{L}[\p{L}\p{M}\p{Pd}\p{Pc}\']*")

def tokenize(text: str, cfg: Dict) -> List[str]:
    s = text
    if cfg.get("nfkc", True):
        import unicodedata as ud
        s = ud.normalize("NFKC", s)
    if cfg.get("lower", True):
        s = s.lower()
    if cfg.get("remove_digits", False):
        s = re.sub(r"\p{N}+", " ", s)
    toks = WORD_RE.findall(s)
    if cfg.get("split_hyphens", False):
        spl = []
        for t in toks:
            spl.extend(t.split("-"))
        toks = [t for t in spl if t]
    # FW-aware split (very light touch)
    if cfg is CONFIGS["generous"]:
        g = []
        for t in toks:
            g.extend(re.sub(r"(?<=[\p{Ll}])(?=[\p{Lu}])", " ", t).split())
        toks = g
    mlen = cfg.get("min_len", 1)
    toks = [t for t in toks if len(t) >= mlen]
    return toks

def load_processed(name: str) -> str:
    p = DATA_PROC / f"{name}.txt"
    return p.read_text(encoding="utf-8")

TEXTS = {"FW": load_processed("fw"), "Ulysses": load_processed("ulysses")}
print("Loaded texts:", {k: f"{len(v):,} chars" for k,v in TEXTS.items()})

Loaded texts: {'FW': '1,304,732 chars', 'Ulysses': '1,516,830 chars'}


In [8]:
for name, text in TEXTS.items():
    toks_can = tokenize(text, CONFIGS["canonical"])
    toks_gen = tokenize(text, CONFIGS["generous"])
    print(f"{name}: canonical types={len(set(toks_can)):,}  generous types={len(set(toks_gen)):,}")

FW: canonical types=58,083  generous types=58,083
Ulysses: canonical types=29,375  generous types=29,375


**Zipf: discrete MLE, KS, bootstrap, alternatives**

In [11]:
# make "generous" vs "canonical"
import regex as re, unicodedata as ud

CONFIGS = {
    "conservative": dict(lower=False, nfkc=True, split_hyphens=False, remove_digits=False, min_len=1,
                         fw_split_camel=False, generous_punct=False),
    "canonical":    dict(lower=True,  nfkc=True, split_hyphens=True,  remove_digits=True,  min_len=1,
                         fw_split_camel=False, generous_punct=False),
    "generous":     dict(lower=True,  nfkc=True, split_hyphens=True,  remove_digits=True,  min_len=1,
                         fw_split_camel=True,  generous_punct=True),
}

WORD_RE = re.compile(r"\p{L}[\p{L}\p{M}\p{Pd}\p{Pc}’']*")

GENEROUS_SPLIT_CHARS = "[/·\u00B7\u2012-\u2015\u2212]"

def tokenize(text: str, cfg: dict):
    s = text

    if cfg.get("nfkc", True):
        s = ud.normalize("NFKC", s)

    # (generous) split on punctuation joiners
    if cfg.get("generous_punct", False):
        s = re.sub(GENEROUS_SPLIT_CHARS, " ", s)

    # (generous) split camelCase
    if cfg.get("fw_split_camel", False):
        s = re.sub(r"(?<=[\p{Ll}])(?=[\p{Lu}])", " ", s)

    if cfg.get("lower", True):
        s = s.lower()
    if cfg.get("remove_digits", False):
        s = re.sub(r"\p{N}+", " ", s)

    # Extract word tokens
    toks = WORD_RE.findall(s)

    # Opt split on hyphens
    if cfg.get("split_hyphens", False):
        tt = []
        for t in toks:
            tt.extend(t.split("-"))
        toks = [t for t in tt if t]

    # Length filter
    mlen = cfg.get("min_len", 1)
    return [t for t in toks if len(t) >= mlen]


In [12]:
for name, text in TEXTS.items():
    toks_can = tokenize(text, CONFIGS["canonical"])
    toks_gen = tokenize(text, CONFIGS["generous"])
    print(f"{name}: canonical types={len(set(toks_can)):,}  generous types={len(set(toks_gen)):,}")

FW: canonical types=58,083  generous types=58,019
Ulysses: canonical types=30,187  generous types=30,158


In [13]:
# Core: discrete power-law, Hurwitz zeta via mpmath
import mpmath as mp
from scipy.optimize import minimize_scalar, minimize
from numpy.random import default_rng
from scipy.stats import norm

mp.mp.dps = 50  # high precision for zeta

def hurwitz_zeta(alpha: float, xmin: int) -> float:
    return float(mp.zeta(alpha, xmin))

def loglik_discrete_powerlaw(xs: np.ndarray, alpha: float, xmin: int) -> float:
    if alpha <= 1: return -np.inf
    Z = hurwitz_zeta(alpha, xmin)
    if not np.isfinite(Z) or Z <= 0: return -np.inf
    return - xs.size * math.log(Z) - alpha * np.log(xs).sum()

def mle_alpha_discrete(xs: np.ndarray, xmin: int) -> float:
    def nll(a): return -loglik_discrete_powerlaw(xs, a, xmin)
    res = minimize_scalar(nll, bounds=(1.0001, 5.0), method="bounded")
    return float(res.x)

def cdf_discrete_powerlaw_support(alpha: float, xmin: int, xmax: int) -> np.ndarray:
    # returns CDF over support [xmin..xmax]
    ks = np.arange(xmin, xmax+1, dtype=int)
    w = ks.astype(float)**(-alpha)
    W = w.cumsum()
    return W / W[-1]

def ks_statistic(xs: np.ndarray, alpha: float, xmin: int) -> float:
    # xs: tail sample (>= xmin)
    xs = np.sort(xs)
    uniq, counts = np.unique(xs, return_counts=True)
    ecdf = np.cumsum(counts) / xs.size
    F = cdf_discrete_powerlaw_support(alpha, xmin, uniq.max())
    # align CDF values to uniq indices
    # F[k - xmin] is CDF at k
    model = F[uniq - xmin]
    return float(np.max(np.abs(ecdf - model)))

def choose_xmin_by_ks(counts: np.ndarray, xmin_candidates: List[int]) -> Tuple[int, float, float]:
    """Return (xmin, alpha, ksD) minimizing KS distance."""
    best = (None, None, np.inf)
    for xm in xmin_candidates:
        tail = counts[counts >= xm]
        if tail.size < 50:  # safety
            continue
        a = mle_alpha_discrete(tail, xm)
        D = ks_statistic(tail, a, xm)
        if D < best[2]:
            best = (xm, a, D)
    if best[0] is None:
        raise ValueError("No valid xmin candidate produced sufficient tail.")
    return best  # xmin, alpha, D

def parametric_ks_bootstrap(counts: np.ndarray, alpha: float, xmin: int, B: int = 800, seed: int = 42) -> float:
    """KS p-value via parametric bootstrap: synthetic samples from fitted model (tail size preserved)."""
    rng = default_rng(seed)
    tail = counts[counts >= xmin]
    n = tail.size
    # Build tail pmf over [xmin..K] with mass cutoff
    K = max(int(tail.max()*3), xmin+1000)
    ks = np.arange(xmin, K+1)
    w = ks.astype(float)**(-alpha)
    pmf = w / w.sum()
    D_obs = ks_statistic(tail, alpha, xmin)
    ge = 0
    for _ in range(B):
        sim = rng.choice(ks, size=n, replace=True, p=pmf)
        a_sim = mle_alpha_discrete(sim, xmin)
        D_sim = ks_statistic(sim, a_sim, xmin)
        if D_sim >= D_obs:
            ge += 1
    return (ge + 1) / (B + 1)

def bootstrap_alpha_ci(counts: np.ndarray, xmin: int, B: int = 800, seed: int = 7) -> Tuple[float, float]:
    rng = default_rng(seed)
    tail = counts[counts >= xmin]
    n = tail.size
    alphas = np.empty(B, float)
    for b in range(B):
        samp = rng.choice(tail, size=n, replace=True)
        alphas[b] = mle_alpha_discrete(samp, xmin)
    return float(np.percentile(alphas, 2.5)), float(np.percentile(alphas, 97.5))

**Alternative models + ML estimates (for LR tests)**

In [14]:
# Lognormal (discretised), truncated power law, Zipf–Mandelbrot
def fit_lognormal_tail(xs: np.ndarray, xmin: int) -> Tuple[float, float]:
    x = np.log(xs)
    mu = float(x.mean()); sigma = float(x.std(ddof=1))
    return mu, max(sigma, 1e-6)

def fit_truncated_pl_tail(xs: np.ndarray, xmin: int) -> Tuple[float, float]:
    xs = xs.astype(float)
    def nll(params):
        alpha, lam = params
        if alpha <= 0 or lam <= 0: return 1e12
        # normalisation over support
        K = max(int(xs.max()*3), xmin+1000)
        ks = np.arange(xmin, K+1, dtype=float)
        Z = np.sum(ks**(-alpha) * np.exp(-lam*ks))
        if not np.isfinite(Z) or Z <= 0: return 1e12
        ll = -np.sum(alpha*np.log(xs) + lam*xs + np.log(Z))
        return -ll
    res = minimize(nll, x0=np.array([1.5, 1e-3]), bounds=[(1e-6, 5.0), (1e-9, 1.0)])
    return float(res.x[0]), float(res.x[1])

def fit_zipf_mandelbrot_tail(xs: np.ndarray, xmin: int) -> Tuple[float, float]:
    # p(k) ∝ (k+q)^(-s) with s>0, q≥0
    def nll(params):
        s, q = params
        if s <= 0 or q < 0: return 1e12
        K = max(int(xs.max()*3), xmin+1000)
        ks = np.arange(xmin, K+1, dtype=float)
        Z = np.sum((ks + q)**(-s))
        if not np.isfinite(Z) or Z <= 0: return 1e12
        ll = -np.sum(s*np.log(xs + q) + np.log(Z))
        return -ll
    res = minimize(nll, x0=np.array([1.2, 1.0]), bounds=[(1e-6, 10.0), (0.0, 100.0)])
    return float(res.x[0]), float(res.x[1])

def lr_test_same_support(xs: np.ndarray, xmin: int, logpmfA, logpmfB) -> Tuple[float, float, float]:
    """Return (LR, z, p) for Vuong. xs are tail counts >= xmin."""
    xs = xs[xs >= xmin]
    lA = logpmfA(xs); lB = logpmfB(xs)
    diff = lA - lB
    LR = float(diff.sum())
    sd = float(np.std(diff, ddof=1))
    z = LR / (sd * math.sqrt(len(diff)) + 1e-12)
    p = 2*(1 - norm.cdf(abs(z)))
    return LR, z, p

def make_logpmfs_for_tail(xmin: int, params: Dict[str, float], model: str):
    # returns a vectorised logpmf(x_array)
    if model == "powerlaw":
        alpha = params["alpha"]
        K = None  # use Hurwitz zeta exact normaliser
        def logpmf(x):
            Z = hurwitz_zeta(alpha, xmin)
            return -alpha*np.log(x) - np.log(Z)
        return np.vectorize(logpmf)
    if model == "lognormal":
        mu, sigma = params["mu"], params["sigma"]
        K = None
        def logpmf(x):
            # discretised via PDF approx, renormed
            x = x.astype(float)
            pdf = (1/(x*sigma*math.sqrt(2*math.pi)))*np.exp(-(np.log(x)-mu)**2/(2*sigma**2))
            # renormalise on tail
            Kmax = int(max(x.max()*3, xmin+1000))
            ks = np.arange(xmin, Kmax+1, dtype=float)
            denom = (1/(ks*sigma*math.sqrt(2*math.pi)))*np.exp(-(np.log(ks)-mu)**2/(2*sigma**2))
            denom = denom.sum()
            return np.log(pdf) - np.log(denom + 1e-300)
        return logpmf
    if model == "truncated":
        alpha, lam = params["alpha"], params["lam"]
        def logpmf(x):
            Kmax = int(max(x.max()*3, xmin+1000))
            ks = np.arange(xmin, Kmax+1, dtype=float)
            denom = np.sum((ks**(-alpha))*np.exp(-lam*ks))
            return -alpha*np.log(x) - lam*x - np.log(denom + 1e-300)
        return logpmf
    if model == "zipf_mandelbrot":
        s, q = params["s"], params["q"]
        def logpmf(x):
            Kmax = int(max(x.max()*3, xmin+1000))
            ks = np.arange(xmin, Kmax+1, dtype=float)
            denom = np.sum((ks + q)**(-s))
            return -s*np.log(x + q) - np.log(denom + 1e-300)
        return logpmf
    raise ValueError("Unknown model")

**Fit wrapper (per text × pipeline) + exports**

In [15]:
def counts_from_tokens(tokens: List[str]) -> np.ndarray:
    return np.array(list(Counter(tokens).values()), dtype=int)

def xmin_candidates_for(counts: np.ndarray, xmin_min: int = 2, max_unique: int = 500) -> np.ndarray:
    u = np.unique(counts)
    u = u[u >= xmin_min]
    return u[:max_unique]

def fit_all_models(counts: np.ndarray, xmin: int, alpha_hat: float) -> pd.DataFrame:
    tail = counts[counts >= xmin]
    rows = []
    # powerlaw
    rows.append(dict(model="powerlaw", alpha=alpha_hat))
    # lognormal
    mu, sigma = fit_lognormal_tail(tail, xmin)
    rows.append(dict(model="lognormal", mu=mu, sigma=sigma))
    # truncated
    a_t, lam = fit_truncated_pl_tail(tail, xmin)
    rows.append(dict(model="truncated", alpha=a_t, lam=lam))
    # zipf-mandelbrot
    s, q = fit_zipf_mandelbrot_tail(tail, xmin)
    rows.append(dict(model="zipf_mandelbrot", s=s, q=q))
    return pd.DataFrame(rows)

def compare_models_lr(counts: np.ndarray, xmin: int, param_table: pd.DataFrame) -> pd.DataFrame:
    tail = counts[counts >= xmin]
    # build logpmfs
    lp = {}
    for _, row in param_table.iterrows():
        model = row["model"]
        params = {k: row[k] for k in row.index if k not in ["model"] and not pd.isna(row[k])}
        lp[model] = make_logpmfs_for_tail(xmin, params, model)
    # pairwise vs powerlaw
    out = []
    for other in ["lognormal", "truncated", "zipf_mandelbrot"]:
        LR, z, p = lr_test_same_support(tail, xmin, lp["powerlaw"], lp[other])
        out.append(dict(comp=f"powerlaw_vs_{other}", LR=LR, z=z, p=p))
    return pd.DataFrame(out)

def heaps_curve(tokens: List[str], steps: int = 200) -> pd.DataFrame:
    N = len(tokens)
    step = max(1000, N // steps)
    vocab = set()
    Ns, Vs = [], []
    for i in range(step, N+1, step):
        for t in tokens[i-step:i]:
            vocab.add(t)
        Ns.append(i); Vs.append(len(vocab))
    df = pd.DataFrame(dict(N=Ns, V=Vs))
    # Estimate beta by OLS on log-log (descriptive)
    x = np.log(df["N"].values)
    y = np.log(df["V"].values + 1e-9)
    beta, logK = np.polyfit(x, y, 1)
    df.attrs["beta_hat"] = float(beta)
    df.attrs["K_hat"] = float(np.exp(logK))
    return df

def bootstrap_heaps_beta(tokens: List[str], B: int = 400, steps: int = 200, seed: int = 11) -> Tuple[float,float]:
    rng = default_rng(seed)
    Ns = len(tokens)
    betas = np.empty(B, float)
    for b in range(B):
        # block bootstrap: resample contiguous blocks (approximate dependence)
        block = max(1000, Ns // steps)
        idx = []
        while len(idx) < Ns:
            start = int(rng.integers(0, max(1, Ns - block)))
            idx.extend(range(start, min(Ns, start + block)))
        idx = idx[:Ns]
        sample = [tokens[i] for i in idx]
        hc = heaps_curve(sample, steps=steps)
        betas[b] = hc.attrs["beta_hat"]
    return float(np.percentile(betas, 2.5)), float(np.percentile(betas, 97.5))

def windowed_s(tokens: List[str], window: int = 25000, step: int = 5000, xmin_min: int = 2) -> pd.DataFrame:
    rows = []
    for start in range(0, max(1, len(tokens) - window + 1), step):
        seg = tokens[start:start+window]
        counts = counts_from_tokens(seg)
        xmins = xmin_candidates_for(counts, xmin_min=xmin_min, max_unique=300)
        try:
            xmin, alpha, D = choose_xmin_by_ks(counts, xmins.tolist())
        except Exception:
            continue
        n_tail = int((counts >= xmin).sum())
        rows.append(dict(window_start=start, window_end=start+window, xmin=xmin, s_hat=alpha, ks_D=D, n_tail=n_tail))
    return pd.DataFrame(rows)

**fits for each pipeline; export CSVs**

In [16]:
ALL_FITS = []
ALL_LR   = []
ALL_HEAPS = []
ALL_RANKF = []
ALL_WINS = []

for pipe, cfg in CONFIGS.items():
    print(f"\n=== PIPELINE: {pipe} ===")
    for text_name, text in TEXTS.items():
        print(f"\n→ Tokenising {text_name} …")
        tokens = tokenize(text, cfg)
        counts = counts_from_tokens(tokens)
        print(f"{text_name}: tokens={len(tokens):,}  types={len(counts):,}")
        # xmin selection + alpha
        xmins = xmin_candidates_for(counts, xmin_min=2, max_unique=500)
        xmin, alpha, D = choose_xmin_by_ks(counts, xmins.tolist())
        n_tail = int((counts >= xmin).sum())
        print(f"{text_name}: xmin={xmin}  alpha={alpha:.4f}  KS D={D:.4f}  tail_n={n_tail:,}")
        # KS bootstrap + alpha CI
        ks_p = parametric_ks_bootstrap(counts, alpha, xmin, B=800, seed=42)
        ci_lo, ci_hi = bootstrap_alpha_ci(counts, xmin, B=800, seed=7)
        # store headline row
        ALL_FITS.append(dict(text=text_name, pipeline=pipe, model="powerlaw",
                             s=alpha, s_ci_low=ci_lo, s_ci_high=ci_hi,
                             x_min=xmin, ks_D=D, ks_p=ks_p,
                             tokens=len(tokens), types=len(counts), n_tail=n_tail))
        # Alternatives + LR
        params = fit_all_models(counts, xmin, alpha)
        lr = compare_models_lr(counts, xmin, params)
        lr["text"] = text_name; lr["pipeline"] = pipe; lr["xmin"] = xmin
        ALL_LR.append(lr)
        # Heaps
        hc = heaps_curve(tokens, steps=240)
        beta = hc.attrs["beta_hat"]; Khat = hc.attrs["K_hat"]
        beta_lo, beta_hi = bootstrap_heaps_beta(tokens, B=300, steps=180, seed=13)
        ALL_HEAPS.append(pd.DataFrame({
            "text":[text_name], "pipeline":[pipe], "beta_hat":[beta],
            "beta_ci_low":[beta_lo], "beta_ci_high":[beta_hi],
            "K_hat":[Khat]
        }))
        # rank-frequency (for dashboard)
        freqs = np.array(sorted(counts, reverse=True))
        rf = pd.DataFrame({"text":text_name, "pipeline":pipe,
                           "rank":np.arange(1, freqs.size+1), "freq":freqs})
        ALL_RANKF.append(rf)
        # windowed s (can be time-consuming; tweak window/step as needed)
        print(f"{text_name}: windowed s …")
        wins = windowed_s(tokens, window=25000, step=5000, xmin_min=2)
        wins["text"] = text_name; wins["pipeline"] = pipe
        ALL_WINS.append(wins)

# Concatenate & export
fits_df = pd.DataFrame(ALL_FITS).sort_values(["text","pipeline"]).reset_index(drop=True)
lr_df   = pd.concat(ALL_LR, ignore_index=True)
heaps_df= pd.concat(ALL_HEAPS, ignore_index=True)
rf_df   = pd.concat(ALL_RANKF, ignore_index=True)
wins_df = pd.concat(ALL_WINS, ignore_index=True)

fits_df.to_csv(TABLES/"model_fits.csv", index=False)
lr_df.to_csv(TABLES/"model_comparisons.csv", index=False)
heaps_df.to_csv(TABLES/"heaps.csv", index=False)
rf_df.to_csv(TABLES/"rank_freq.csv", index=False)
wins_df.to_csv(TABLES/"window_s.csv", index=False)

headline = fits_df.rename(columns={"s":"alpha"})
headline[["text","pipeline","alpha","s_ci_low","s_ci_high","x_min","ks_p","tokens","types","n_tail"]]\
        .to_csv(TABLES/"headline_stats.csv", index=False)

print("\nSaved tables:")
for p in sorted(TABLES.glob("*.csv")):
    print(" -", p.name, p.stat().st_size, "bytes")


=== PIPELINE: conservative ===

→ Tokenising FW …
FW: tokens=218,620  types=62,255
FW: xmin=6  alpha=1.9963  KS D=0.0143  tail_n=2,642
FW: windowed s …

→ Tokenising Ulysses …
Ulysses: tokens=264,231  types=34,597
Ulysses: xmin=4  alpha=1.9545  KS D=0.0059  tail_n=6,867
Ulysses: windowed s …

=== PIPELINE: canonical ===

→ Tokenising FW …
FW: tokens=219,320  types=58,083
FW: xmin=7  alpha=2.0111  KS D=0.0167  tail_n=2,271
FW: windowed s …

→ Tokenising Ulysses …
Ulysses: tokens=264,442  types=30,187
Ulysses: xmin=5  alpha=1.9519  KS D=0.0077  tail_n=5,253
Ulysses: windowed s …

=== PIPELINE: generous ===

→ Tokenising FW …
FW: tokens=219,492  types=58,019
FW: xmin=7  alpha=2.0123  KS D=0.0170  tail_n=2,278
FW: windowed s …

→ Tokenising Ulysses …
Ulysses: tokens=264,554  types=30,158
Ulysses: xmin=5  alpha=1.9518  KS D=0.0079  tail_n=5,256
Ulysses: windowed s …

Saved tables:
 - headline_stats.csv 741 bytes
 - headline_stats_canonical.csv 326 bytes
 - heaps.csv 604 bytes
 - model_comp

In [17]:
import pandas as pd, numpy as np
from pathlib import Path

ROOT = Path("/content/drive/MyDrive/zipf_joyce")
headline = pd.read_csv(ROOT/"results/tables/headline_stats.csv")
# add implied Zipf rank exponent s = 1/(alpha-1)
headline["s_hat"] = 1.0/(headline["s"] - 1.0) if "s" in headline.columns else np.nan
if "alpha" in headline.columns:
    headline["s_hat"] = 1.0/(headline["alpha"] - 1.0)

# show key cols if present
cols = [c for c in ["text","pipeline","alpha","s","s_hat","x_min","ks_D","ks_p","n_tail","tokens","types","s_ci_low","s_ci_high","alpha_ci_low","alpha_ci_high"] if c in headline.columns]
display(headline[cols].sort_values(["text","pipeline"]).reset_index(drop=True))


,text,pipeline,alpha,s_hat,x_min,ks_p,n_tail,tokens,types,s_ci_low,s_ci_high
0,FW,canonical,2.011055,0.989066,7,0.138577,2271,219320,58083,1.968700,2.056860
1,FW,conservative,1.996272,1.003742,6,0.215980,2642,218620,62255,1.959684,2.034336
2,FW,generous,2.012281,0.987868,7,0.123596,2278,219492,58019,1.972522,2.058321
3,Ulysses,canonical,1.951869,1.050565,5,0.544320,5253,264442,30187,1.926414,1.976456
4,Ulysses,conservative,1.954534,1.047632,4,0.670412,6867,264231,34597,1.933773,1.975303
5,Ulysses,generous,1.951849,1.050587,5,0.490637,5256,264554,30158,1.929386,1.976397


**Static figures**

In [18]:
# Zipf plots (log–log) per text/pipeline
import matplotlib.pyplot as plt

def quick_zipf_plot(df, text, pipeline):
    sub = df[(df["text"]==text)&(df["pipeline"]==pipeline)]
    plt.figure(figsize=(7,5))
    plt.scatter(sub["rank"], sub["freq"], s=6)
    plt.xscale("log"); plt.yscale("log")
    plt.xlabel("Rank (log)"); plt.ylabel("Frequency (log)")
    plt.title(f"Zipf — {text} · {pipeline}")
    out = FIGS / f"zipf_{text}_{pipeline}.png"
    plt.tight_layout(); plt.savefig(out, dpi=180); plt.close()
    return out

outs = []
for t in ["FW","Ulysses"]:
    for p in CONFIGS.keys():
        outs.append(quick_zipf_plot(rf_df, t, p))
print("Static figures written:", [o.name for o in outs[:3]], "…")

Static figures written: ['zipf_FW_conservative.png', 'zipf_FW_canonical.png', 'zipf_FW_generous.png'] …


**Zip and (if Colab) download; also mirror to Drive**

In [19]:
import zipfile
ZIPPATH = RESULTS / "zipf_results_bundle.zip"
with zipfile.ZipFile(ZIPPATH, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for sub in ["tables","figures","dashboard"]:
        d = RESULTS / sub
        if d.exists():
            for f in d.rglob("*"):
                if f.is_file():
                    z.write(f, f.relative_to(RESULTS))
print("Zipped results →", ZIPPATH)

if IN_COLAB:
    files.download(str(ZIPPATH))

Zipped results → /content/drive/MyDrive/zipf_joyce/results/zipf_results_bundle.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Methods: Statistical Model and Inference for Zipf/Power-Law Behavior

## Data and Preprocessing
This project analyses Finnegans Wake (FW) and Ulysses prepared via a reproducible pipeline (UTF-8, NFKC, boilerplate removal, de-hyphenation at line breaks, optional header/page-number stripping). Tokenisation varies across three configurations (“conservative,” “canonical,” “generous”) to assess sensitivity. For each configuration the pipeline computes type counts \(x_i \in \mathbb{N}\) (number of occurrences per type) and operates on the multiset of counts \(\mathcal{X}=\{x_1,\dots,x_n\}\).

**Notation.**  
- \(x_{\min}\): lower cutoff of the heavy-tail region (on counts)  
- \(n_{\text{tail}}=\#\{x_i\in\mathcal{X}: x_i\ge x_{\min}\}\)  
- \(\alpha\): exponent for the discrete power law  
- \(s\): Zipf exponent in the rank domain (used only for visualisation)  
- \(k\): frequency value; \(N_k\): number of types with frequency \(k\)

## Model: Discrete Power Law (Counts Domain)
This project fits the discrete power law to the tail of the count distribution:
$$
p(x\mid \alpha, x_{\min})=\frac{x^{-\alpha}}{\zeta(\alpha,x_{\min})},\qquad
x\in\{x_{\min},x_{\min}{+}1,\dots\},\ \alpha>1,
$$
where \(\zeta(\alpha,x_{\min})\) is the Hurwitz zeta. The tail log-likelihood for counts \(\{x_j\}_{j=1}^{n_{\text{tail}}}\) is
$$
\ell(\alpha; x_{\min})=-\,n_{\text{tail}}\log \zeta(\alpha,x_{\min})
\;-\; \alpha \sum_{j=1}^{n_{\text{tail}}}\log x_j \, .
$$

> **Why counts, not ranks?** Fitting straight lines to log–log rank plots via OLS is biased and yields unreliable uncertainty. This project estimates \(\alpha\) by discrete MLE on counts and keeps rank plots for visualisation only.

## Estimation
1. **Choosing \(x_{\min}\)** (tail onset).  
   The pipeline evaluates a grid \(x_{\min}\in\mathcal{G}\) (unique counts \(\ge 2\)). For each candidate, it estimates \(\hat{\alpha}(x_{\min})\) by MLE (bounded 1-D optimisation over \(\alpha>1\)) and selects \(\hat{x}_{\min}\) that minimises the Kolmogorov–Smirnov (KS) distance between empirical and model CDFs on the tail:
   $$
   D(x_{\min})=\sup_{x\ge x_{\min}}
   \bigl|\hat{F}(x)-F(x\mid \hat{\alpha}(x_{\min}),x_{\min})\bigr| \, .
   $$

2. **Goodness-of-fit (parametric bootstrap).**  
   With \((\hat{\alpha},\hat{x}_{\min})\) fixed, the pipeline generates \(B\) synthetic tails of size \(n_{\text{tail}}\) from \(p(x\mid \hat{\alpha},\hat{x}_{\min})\), re-fits \(\alpha\), and recomputes KS for each; the descriptive p-value is
   $$
   \hat{p}=\frac{1+\#\{D^{(b)}\ge D_{\text{obs}}\}}{B+1} \, .
   $$

3. **Uncertainty in \(\alpha\).**  
   The project reports **bootstrap** 95% CIs for \(\alpha\) by resampling tail counts with replacement and re-estimating \(\alpha\).

4. **Tail size criterion.**  
   The analysis requires \(n_{\text{tail}}\ge 50\) (preferably \(\ge 100\)) for stable inference; smaller tails are flagged and excluded from comparisons.

## Model Comparison on the Same Support
On the same tail support \(\{x\ge \hat{x}_{\min}\}\), this project also fits:
- **Lognormal (discretised)** with \((\mu,\sigma)\)  
- **Truncated power law** \(p(x)\propto x^{-\alpha}e^{-\lambda x}\)  
- **Zipf–Mandelbrot** \(p(x)\propto (x+q)^{-s}\) with \(q\ge 0\)

It uses Vuong’s likelihood-ratio test for non-nested models and reports LR, \(z\), and two-sided \(p\):
$$
\mathrm{LR}=\sum_{j=1}^{n_{\text{tail}}}\bigl[\log p_A(x_j)-\log p_B(x_j)\bigr],\qquad
z=\frac{\mathrm{LR}}{\hat{\sigma}(\Delta)\sqrt{n_{\text{tail}}}} \, .
$$
Effect sizes and uncertainty are prioritised over binary accept/reject decisions.

## Heaps’ Law (Vocabulary Growth)
On cumulative token prefixes \(1{:}N\), the project measures the type–token relation and fits
$$
V(N)\approx K\,N^{\beta}.
$$
It reports \(\hat{\beta}\) with a block-bootstrap CI (resampling contiguous token blocks to respect dependence). Heaps and Zipf need not agree empirically; results are therefore reported separately.

## Windowed Stability of \(\alpha\)
To assess local stability, the pipeline computes \(\hat{\alpha}\) on sliding token windows (size \(W\), step \(S\)) with the same \(x_{\min}\) selection per window and visualises trajectories of \(\hat{\alpha}\) alongside tail sizes \(n_{\text{tail}}\).

## Reporting
For each text × tokenisation the tables include:
- **Tail exponent** \(\hat{\alpha}\) with 95% CI
- **Cutoff** \(\hat{x}_{\min}\), tail size \(n_{\text{tail}}\), KS \(D\), and GoF \(\hat{p}\)  
- **Model comparisons**: LR \(+\) \(z\) \(+\) \(p\) vs lognormal, truncated, Zipf–Mandelbrot  
- **Descriptives**: hapax share, \(N_k\) curve, rank–frequency (visual only)  
- **Heaps**: \(\hat{\beta}\) with CI  
- **Windowed** \(\hat{\alpha}\) series

## Sensitivity and Reproducibility
- **Sensitivity**: the full inference is repeated under three tokenisation regimes; optional size-matching by random subsampling controls finite-size effects.  
- **Reproducibility**: seeds are fixed; edition and preprocessing are recorded; the \(x_{\min}\) grid, optimisation bounds, and bootstrap sizes are logged. Outputs are saved as CSV to `results/tables/` and mirrored to Drive; the HTML dashboard is self-contained.

## Implementation Details
- **Optimisation**: 1-D bounded minimisation for \(\alpha\in(1.0001,5]\); 2-D bounded for Mandelbrot and truncated models. Hurwitz zeta \(\zeta(\alpha,x_{\min})\) is evaluated with high-precision arithmetic.  
- **KS computation**: the model CDF is evaluated on the empirical support \(\{x_{\min},\dots,x_{\max}\}\); KS is the sup norm between empirical and model CDFs.  
- **Bootstrap sizes**: default \(B=800\) for KS GoF and \(\alpha\) CIs; \(B\approx300\) for Heaps’ \(\beta\) block bootstrap.  
- **Exclusions**: fits failing the tail-size criterion or with unstable normalisers are flagged and omitted from cross-text comparisons.

## Limitations
- Tokens are dependent; bootstrap CIs reflect variability of types, not full linguistic dynamics.  
- The discretised lognormal uses a PDF approximation with tail renormalisation (exact integrals are slower but empirically similar here).  
- Windowed estimates fluctuate when \(n_{\text{tail}}\) is small; both \(\hat{\alpha}\) and \(n_{\text{tail}}\) are shown.

## References
- Clauset, Aaron, Cosma Rohilla Shalizi, and M. E. J. Newman. 2009. “Power-Law Distributions in Empirical Data.” *SIAM Review* 51 (4): 661–703.  
- Mitzenmacher, Michael. 2004. “A Brief History of Generative Models for Power Law and Lognormal Distributions.” *Internet Mathematics* 1 (2): 226–251.  
- Baayen, R. Harald. 2001. *Word Frequency Distributions.* Dordrecht: Kluwer.  
- Piantadosi, Steven T. 2014. “Zipf’s Law in Natural Language: A Critical Review.” *Psychonomic Bulletin & Review* 21 (5): 1112–1130.
